In [3]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fairness_functions as ff
import dalex as dx 
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
import pickle
from time import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, ConfusionMatrixDisplay
from sklearn.neural_network import MLPClassifier

plotly.offline.init_notebook_mode()
warnings.filterwarnings("ignore")

In [2]:
# with open("/Users/henryelj/obj_dict_backup_2.pkl", "rb") as file:
#     # Deserialize and load the object from the file
#     obj_dict = pickle.load(file)

# for name, obj in obj_dict.items():
#     exec(name + "= obj")

# globals().update(obj_dict)

In [81]:
feature_dtypes = {
    "respondent_id": "Int64"
    , "h1n1_concern": "Int64"
    , "h1n1_knowledge": "Int64"
    , "behavioral_antiviral_meds": "Int64"
    , "behavioral_avoidance": "Int64"
    , "behavioral_face_mask": "Int64"
    , "behavioral_wash_hands": "Int64"
    , "behavioral_large_gatherings": "Int64"
    , "behavioral_outside_home": "Int64"
    , "behavioral_touch_face": "Int64"
    , "doctor_recc_h1n1": "Int64"
    , "doctor_recc_seasonal": "Int64"
    , "chronic_med_condition": "Int64"
    , "child_under_6_months": "Int64"
    , "health_worker": "Int64"
    , "health_insurance": "Int64"
    , "opinion_h1n1_vacc_effective": "Int64"
    , "opinion_h1n1_risk": "Int64"
    , "opinion_h1n1_sick_from_vacc": "Int64"
    , "opinion_seas_vacc_effective": "Int64"
    , "opinion_seas_risk": "Int64"
    , "opinion_seas_sick_from_vacc": "Int64"
    , "age_group": "category"
    , "education": "category"
    , "race": "category"
    , "sex": "category"
    , "income_poverty": "category"
    , "marital_status": "category"
    , "rent_or_own": "category"
    , "employment_status": "category"
    , "hhs_geo_region": "category"
    , "census_msa": "category"
    , "household_adults": "Int64"
    , "household_children": "Int64"
    , "employment_industry": "category"
    , "employment_occupation": "category"
    }

label_dtypes = {
    "h1n1_vaccine": "Int64"
    , "seasonal_vaccine": "Int64"
    }

df = pd.read_csv("training_set_features.csv", dtype = feature_dtypes).merge(pd.read_csv("training_set_labels.csv", dtype = label_dtypes), how = "left", on = "respondent_id")
df = df.drop("respondent_id", axis = 1)
df = df.apply(lambda x: x.astype("str").replace("nan", "Missing").astype("category") if x.dtypes == "category" else x.fillna(999))

In [4]:
st.title("Investigation into the Distribution of Vaccines")
st.write("Investigating vaccine uptake based on protected, opinion and environmental characteristics")

st.write('''
         Explain further
         ''')

2024-07-11 00:48:21.565 
  command:

    streamlit run /usr/local/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [12]:
data_dictionary = {
    "h1n1_vaccine": {
        "description": "Whether respondent received H1N1 flu vaccine.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "seasonal_vaccine": {
        "description": "Whether respondent received seasonal flu vaccine.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "respondent_id": {
        "description": "Unique and random identifier.",
        "mappings": {
            None
        }
    },
    "h1n1_concern": {
        "description": "Level of concern about the H1N1 flu.",
        "mappings": {
            0: "Not at all concerned",
            1: "Not very concerned",
            2: "Somewhat concerned",
            3: "Very concerned"
        }
    },
    "h1n1_knowledge": {
        "description": "Level of knowledge about H1N1 flu.",
        "mappings": {
            0: "No knowledge",
            1: "A little knowledge",
            2: "A lot of knowledge"
        }
    },
    "behavioral_antiviral_meds": {
        "description": "Has taken antiviral medications.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "behavioral_avoidance": {
        "description": "Has avoided close contact with others with flu-like symptoms.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "behavioral_face_mask": {
        "description": "Has bought a face mask.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "behavioral_wash_hands": {
        "description": "Has frequently washed hands or used hand sanitizer.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "behavioral_large_gatherings": {
        "description": "Has reduced time at large gatherings.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "behavioral_outside_home": {
        "description": "Has reduced contact with people outside of own household.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "behavioral_touch_face": {
        "description": "Has avoided touching eyes, nose, or mouth.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "doctor_recc_h1n1": {
        "description": "H1N1 flu vaccine was recommended by doctor.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "doctor_recc_seasonal": {
        "description": "Seasonal flu vaccine was recommended by doctor.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "chronic_med_condition": {
        "description": "Has any of the listed chronic medical conditions.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "child_under_6_months": {
        "description": "Has regular close contact with a child under the age of six months.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "health_worker": {
        "description": "Is a healthcare worker.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "health_insurance": {
        "description": "Has health insurance.",
        "mappings": {
            0: "No",
            1: "Yes"
        }
    },
    "opinion_h1n1_vacc_effective": {
        "description": "Respondent's opinion about H1N1 vaccine effectiveness.",
        "mappings": {
            1: "Not at all effective",
            2: "Not very effective",
            3: "Don't know",
            4: "Somewhat effective",
            5: "Very effective"
        }
    },
    "opinion_h1n1_risk": {
        "description": "Respondent's opinion about risk of getting sick with H1N1 flu without vaccine.",
        "mappings": {
            1: "Very Low",
            2: "Somewhat low",
            3: "Don't know",
            4: "Somewhat high",
            5: "Very high"
        }
    },
    "opinion_h1n1_sick_from_vacc": {
        "description": "Respondent's worry of getting sick from taking H1N1 vaccine.",
        "mappings": {
            1: "Not at all worried",
            2: "Not very worried",
            3: "Don't know",
            4: "Somewhat worried",
            5: "Very worried"
        }
    },
    "opinion_seas_vacc_effective": {
        "description": "Respondent's opinion about seasonal flu vaccine effectiveness.",
        "mappings": {
            1: "Not at all effective",
            2: "Not very effective",
            3: "Don't know",
            4: "Somewhat effective",
            5: "Very effective"
        }
    },
    "opinion_seas_risk": {
        "description": "Respondent's opinion about risk of getting sick with seasonal flu without vaccine.",
        "mappings": {
            1: "Very Low",
            2: "Somewhat low",
            3: "Don't know",
            4: "Somewhat high",
            5: "Very high"
        }
    },
    "opinion_seas_sick_from_vacc": {
        "description": "Respondent's worry of getting sick from taking seasonal flu vaccine.",
        "mappings": {
            1: "Not at all worried",
            2: "Not very worried",
            3: "Don't know",
            4: "Somewhat worried",
            5: "Very worried"
        }
    },
    "age_group": {
        "description": "Age group of respondent.",
        "mappings": {
            None
        }
    },
    "education": {
        "description": "Self-reported education level.",
        "mappings": {
            None
        }
    },
    "race": {
        "description": "Race of respondent.",
        "mappings": {
            None
        }
    },
    "sex": {
        "description": "Sex of respondent.",
        "mappings": {
            None
        }
    },
    "income_poverty": {
        "description": "Household annual income of respondent with respect to 2008 Census poverty thresholds.",
        "mappings": {
            None
        }
    },
    "marital_status": {
        "description": "Marital status of respondent.",
        "mappings": {
            None
        }
    },
    "rent_or_own": {
        "description": "Housing situation of respondent.",
        "mappings": {
            None
        }
    },
    "employment_status": {
        "description": "Employment status of respondent.",
        "mappings": {
            None
        }
    },
    "hhs_geo_region": {
        "description": "Respondent's residence using a 10-region geographic classification defined by the U.S. Dept. of Health and Human Services.",
        "mappings": {
            None
        }
    },
    "census_msa": {
        "description": "Respondent's residence within metropolitan statistical areas (MSA) as defined by the U.S. Census.",
        "mappings": {
            None
        }
    },
    "household_adults": {
        "description": "Number of other adults in household, top-coded to 3.",
        "mappings": {
            None
        }
    },
    "household_children": {
        "description": "Number of children in household, top-coded to 3.",
        "mappings": {
            None
        }
    },
    "employment_industry": {
        "description": "Type of industry respondent is employed in.",
        "mappings": {
            None
        }
    },
    "employment_occupation": {
        "description": "Type of occupation of respondent.",
        "mappings": {
            None
        }
    }
}

In [10]:
def get_info(var):
    string = "Definition: " + data_dictionary[var]["description"] + (" | Mappings: " + str(data_dictionary[var]["mappings"]) if data_dictionary[var]["mappings"] != {None} else "")
    return string

In [52]:
# Sidebar with options
st.sidebar.header("Filter Options")

# Target Feature
st.sidebar.write("Target Features")
h1n1_vaccine        = st.sidebar.multiselect("H1N1 Vaccine",        np.unique(df["h1n1_vaccine"].dropna()),     np.unique(df["h1n1_vaccine"].dropna()),     help = get_info("h1n1_vaccine"))
seasonal_vaccine    = st.sidebar.multiselect("Seasonal Vaccine",    np.unique(df["seasonal_vaccine"].dropna()), np.unique(df["seasonal_vaccine"].dropna()), help = get_info("seasonal_vaccine"))

# age = st.slider("Age", int(df["Age"].min()), int(df["Age"].max()), (int(df["Age"].min()), int(df["Age"].max())), help = var_desc["Age"])
# sex = st.multiselect("Sex", df["Sex"].unique(), df["Sex"].unique(), help = get_info("Sex"))

# Protected Characteristics
protected_chars = st.sidebar.expander("Protected Characteristics")
with protected_chars:
    age_group               = st.multiselect("Age Group",                   df["age_group"].unique().astype(str),               df["age_group"].unique().astype(str),               help = get_info("age_group"))
    education               = st.multiselect("Education",                   df["education"].unique().astype(str),               df["education"].unique().astype(str),               help = get_info("education"))
    race                    = st.multiselect("Race",                        df["race"].unique().astype(str),                    df["race"].unique().astype(str),                    help = get_info("race"))
    sex                     = st.multiselect("Sex",                         df["sex"].unique().astype(str),                     df["sex"].unique().astype(str),                     help = get_info("sex"))
    chronic_med_condition   = st.multiselect("Chronic Medical Condition",   np.unique(df["chronic_med_condition"].dropna()),    np.unique(df["chronic_med_condition"].dropna()),    help = get_info("chronic_med_condition"))
    income_poverty          = st.multiselect("Income Poverty",              df["income_poverty"].unique().astype(str),          df["income_poverty"].unique().astype(str),          help = get_info("income_poverty"))

opinion_chars = st.sidebar.expander("Opinion Characteristics")
with opinion_chars:
    opinion_h1n1_vacc_effective = st.multiselect("H1N1 Effective",      np.unique(df["opinion_h1n1_vacc_effective"].dropna()),  np.unique(df["opinion_h1n1_vacc_effective"].dropna()),  help = get_info("opinion_h1n1_vacc_effective"))
    opinion_h1n1_risk           = st.multiselect("H1N1 Risk",           np.unique(df["opinion_h1n1_risk"].dropna()),            np.unique(df["opinion_h1n1_risk"].dropna()),            help = get_info("opinion_h1n1_risk"))
    opinion_h1n1_sick_from_vacc = st.multiselect("H1N1 Sick",           np.unique(df["opinion_h1n1_sick_from_vacc"].dropna()),  np.unique(df["opinion_h1n1_sick_from_vacc"].dropna()),  help = get_info("opinion_h1n1_sick_from_vacc"))
    opinion_seas_vacc_effective = st.multiselect("Seasonal Effective",  np.unique(df["opinion_seas_vacc_effective"].dropna()),  np.unique(df["opinion_seas_vacc_effective"].dropna()),  help = get_info("opinion_seas_vacc_effective"))
    opinion_seas_risk           = st.multiselect("Seasonal Risk",       np.unique(df["opinion_seas_risk"].dropna()),            np.unique(df["opinion_seas_risk"].dropna()),            help = get_info("opinion_seas_risk"))
    opinion_seas_sick_from_vacc = st.multiselect("Seasonal Sick",       np.unique(df["opinion_seas_sick_from_vacc"].dropna()),  np.unique(df["opinion_seas_sick_from_vacc"].dropna()),  help = get_info("opinion_seas_sick_from_vacc"))

behavioural_chars = st.sidebar.expander("Behavioural Characteristics")
with behavioural_chars:
    behavioral_antiviral_meds   = st.multiselect("Antiviral Medication",    np.unique(df["behavioral_antiviral_meds"].dropna()),    np.unique(df["behavioral_antiviral_meds"].dropna()),    help = get_info("behavioral_antiviral_meds"))
    behavioral_avoidance        = st.multiselect("Avoidance",               np.unique(df["behavioral_avoidance"].dropna()),         np.unique(df["behavioral_avoidance"].dropna()),         help = get_info("behavioral_avoidance"))
    behavioral_face_mask        = st.multiselect("Face Mask",               np.unique(df["behavioral_face_mask"].dropna()),         np.unique(df["behavioral_face_mask"].dropna()),         help = get_info("behavioral_face_mask"))
    behavioral_wash_hands       = st.multiselect("Wash Hande",              np.unique(df["behavioral_wash_hands"].dropna()),        np.unique(df["behavioral_wash_hands"].dropna()),        help = get_info("behavioral_wash_hands"))
    behavioral_large_gatherings = st.multiselect("Large Gatherings",        np.unique(df["behavioral_large_gatherings"].dropna()),  np.unique(df["behavioral_large_gatherings"].dropna()),  help = get_info("behavioral_large_gatherings"))
    behavioral_outside_home     = st.multiselect("Outside Home",            np.unique(df["behavioral_outside_home"].dropna()),      np.unique(df["behavioral_outside_home"].dropna()),      help = get_info("behavioral_outside_home"))
    behavioral_touch_face       = st.multiselect("Touch Face",              np.unique(df["behavioral_touch_face"].dropna()),        np.unique(df["behavioral_touch_face"].dropna()),        help = get_info("behavioral_touch_face"))
    
health_chars = st.sidebar.expander("Health Characteristics")
with health_chars:
    h1n1_concern            = st.multiselect("H1N1 Concern",        np.unique(df["h1n1_concern"].dropna()),         np.unique(df["h1n1_concern"].dropna()),         help = get_info("h1n1_concern"))
    h1n1_knowledge          = st.multiselect("H1N1 Knowledge",      np.unique(df["h1n1_knowledge"].dropna()),       np.unique(df["h1n1_knowledge"].dropna()),       help = get_info("h1n1_knowledge"))
    doctor_recc_h1n1        = st.multiselect("Recommend H1N1",      np.unique(df["doctor_recc_h1n1"].dropna()),     np.unique(df["doctor_recc_h1n1"].dropna()),     help = get_info("doctor_recc_h1n1"))
    doctor_recc_seasonal    = st.multiselect("Recommend Seasonal",  np.unique(df["doctor_recc_seasonal"].dropna()), np.unique(df["doctor_recc_seasonal"].dropna()), help = get_info("doctor_recc_seasonal"))
    health_worker           = st.multiselect("Health Worker",       np.unique(df["health_worker"].dropna()),        np.unique(df["health_worker"].dropna()),        help = get_info("health_worker"))
    health_insurance        = st.multiselect("Health Insurance",    np.unique(df["health_insurance"].dropna()),     np.unique(df["health_insurance"].dropna()),     help = get_info("health_insurance"))

# Other Characteristics
other_chars = st.sidebar.expander("Other Characteristics")
with other_chars:
    child_under_6_months    = st.multiselect("Young Child",             np.unique(df["child_under_6_months"].dropna()),     np.unique(df["child_under_6_months"].dropna()),                                                                     help = get_info("child_under_6_months"))
    marital_status          = st.multiselect("Marital Status",          df["marital_status"].unique().astype(str),          df["marital_status"].unique().astype(str),                                                                          help = get_info("marital_status"))
    rent_or_own             = st.multiselect("Rent or Own",             df["rent_or_own"].unique().astype(str),             df["rent_or_own"].unique().astype(str),                                                                             help = get_info("rent_or_own"))
    employment_status       = st.multiselect("Employment Status",       df["employment_status"].unique().astype(str),       df["employment_status"].unique().astype(str),                                                                       help = get_info("employment_status"))
    hhs_geo_region          = st.multiselect("HHS Geo Region",          df["hhs_geo_region"].unique().astype(str),          df["hhs_geo_region"].unique().astype(str),                                                                          help = get_info("hhs_geo_region"))
    census_msa              = st.multiselect("Census MSA",              df["census_msa"].unique().astype(str),              df["census_msa"].unique().astype(str),                                                                              help = get_info("census_msa"))
    household_adults        = st.slider("Household Adults",             int(df["household_adults"].min()),                  int(df["household_adults"].max()), (int(df["household_adults"].min()), int(df["household_adults"].max())),          help = get_info("household_adults"))
    household_children      = st.slider("Household Children",           int(df["household_children"].min()),                int(df["household_children"].max()), (int(df["household_children"].min()), int(df["household_children"].max())),    help = get_info("household_children"))
    employment_industry     = st.multiselect("Employment Industry",     df["employment_industry"].unique().astype(str),     df["employment_industry"].unique().astype(str),                                                                     help = get_info("employment_industry"))
    employment_occupation   = st.multiselect("Employment Occupation",   df["employment_occupation"].unique().astype(str),   df["employment_occupation"].unique().astype(str),                                                                   help = get_info("employment_occupation"))

In [38]:
doctor_recc_h1n1

[0]

0        False
1        False
2        False
3        False
4        False
         ...  
26702    False
26703    False
26704    False
26705    False
26706    False
Name: h1n1_vaccine, Length: 26707, dtype: boolean

In [8]:
filtered_df = df[
    # Target
    (df["h1n1_vaccine"].isin(h1n1_vaccine)) &
    (df["seasonal_vaccine"].isin(seasonal_vaccine)) &
    # Protected Characteristics
    (df["age_group"].isin(age_group)) &
    (df["education"].isin(education)) &
    (df["race"].isin(race)) &
    (df["sex"].isin(sex)) &
    (df["chronic_med_condition"].isin(chronic_med_condition)) &
    (df["income_poverty"].isin(income_poverty)) &
    # Opinion Characteristics
    (df["opinion_h1n1_vacc_effective"].isin(opinion_h1n1_vacc_effective)) &
    (df["opinion_h1n1_risk"].isin(opinion_h1n1_risk)) &
    (df["opinion_h1n1_sick_from_vacc"].isin(opinion_h1n1_sick_from_vacc)) &
    (df["opinion_seas_vacc_effective"].isin(opinion_seas_vacc_effective)) &
    (df["opinion_seas_risk"].isin(opinion_seas_risk)) &
    (df["opinion_seas_sick_from_vacc"].isin(opinion_seas_sick_from_vacc)) &
    # Behavioural Characteristics
    (df["behavioral_antiviral_meds"].isin(behavioral_antiviral_meds)) &
    (df["behavioral_avoidance"].isin(behavioral_avoidance)) &
    (df["behavioral_face_mask"].isin(behavioral_face_mask)) &
    (df["behavioral_wash_hands"].isin(behavioral_wash_hands)) &
    (df["behavioral_large_gatherings"].isin(behavioral_large_gatherings)) &
    (df["behavioral_outside_home"].isin(behavioral_outside_home)) &
    (df["behavioral_touch_face"].isin(behavioral_touch_face)) &
    # Health Characteristics
    (df["h1n1_concern"].isin(h1n1_concern)) &
    (df["h1n1_knowledge"].isin(h1n1_knowledge)) &
    (df["doctor_recc_h1n1"].isin(doctor_recc_h1n1)) &
    (df["doctor_recc_seasonal"].isin(doctor_recc_seasonal)) &
    (df["health_worker"].isin(health_worker)) &
    (df["health_insurance"].isin(health_insurance)) &
    # Other Characteristics
    (df["child_under_6_months"].isin(child_under_6_months)) &
    (df["marital_status"].isin(marital_status)) &
    (df["rent_or_own"].isin(rent_or_own)) &
    (df["employment_status"].isin(employment_status)) &
    (df["hhs_geo_region"].isin(hhs_geo_region)) &
    (df["census_msa"].isin(census_msa)) &
    (df["household_adults"] >= household_adults[0]) & (df["household_adults"] <= household_adults[1]) &
    (df["household_children"] >= household_children[0]) & (df["household_children"] <= household_children[1]) &
    (df["employment_industry"].isin(employment_industry)) &
    (df["employment_industry"].isin(employment_industry))
]

In [9]:
# Display the dataset
st.write("### Inspecting the Data")
st.dataframe(filtered_df)

DeltaGenerator()

In [ ]:
data.mask(df.isin([999, "Missing"])).describe().drop("count", axis = 0).T

In [10]:
def prettyDescribe(data, round_n = 2):
    describe_data = data.mask(data.isin([999, "Missing"])).describe().drop("count", axis = 0).T
    describe_data["missing"] = data.mask(data.isin([999, "Missing"])).isna().agg(sum)
    describe_data["missing%"] = describe_data["missing"] / len(df)
    return round(describe_data, round_n)

In [11]:
st.write(f"{len(filtered_df):,} observations | {len(df) - len(filtered_df):,} ({100 * (len(df) - len(filtered_df))/len(df):.1f}%) of observations filtered out")
st.dataframe(prettyDescribe(filtered_df))

DeltaGenerator()

In [12]:
st.write("### Distributions of Vaccination Indicators")

responses = ["h1n1_vaccine", "seasonal_vaccine"]
features = [x for x in df.columns if x not in ["respondent_id", "h1n1_vaccine", "seasonal_vaccine"]]

tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8, tab9, tab10, tab11, tab12, tab13, tab14, tab15, tab16, tab17, tab18, tab19, tab20, tab21, tab22, tab23, tab24, tab25, tab26, tab27, tab28, tab29, tab30, tab31, tab32, tab33, tab34, tab35 = st.tabs(features)
tabs = [tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8, tab9, tab10, tab11, tab12, tab13, tab14, tab15, tab16, tab17, tab18, tab19, tab20, tab21, tab22, tab23, tab24, tab25, tab26, tab27, tab28, tab29, tab30, tab31, tab32, tab33, tab34, tab35]

In [83]:
def generate_plot_data(feature, target, filtered = False):
    if filtered:
        to_plot = filtered_df[[target, feature]].sort_values(by = [target, feature], ascending = True)
    else:
        to_plot = df[[target, feature]].sort_values(by = [target, feature], ascending = True)
    to_plot = to_plot[~to_plot[feature].isin([999, "Missing"])]
    to_plot = to_plot.value_counts(subset = [target, feature]).to_frame().reset_index()
    to_plot["percent"] = to_plot["count"] / to_plot.groupby(feature)["count"].transform("sum")
    return to_plot.sort_values(by = [target, feature], ascending = True).reset_index(drop = True)

update_legend = {"0": "No", "1": "Yes"}

In [85]:
generate_plot_data("h1n1_concern", "h1n1_vaccine", filtered = False)

h1n1_vaccine  h1n1_concern  count   percent
0             0             0   2849  0.864381
1             0             1   6756  0.828652
2             0             2   8102  0.766147
3             0             3   3250  0.707907
4             1             0    447  0.135619
5             1             1   1397  0.171348
6             1             2   2473  0.233853
7             1             3   1341  0.292093

In [14]:
data_distributions = {}

In [15]:
for feature in features:
    print(feature)
    fig = make_subplots(rows = 2, cols = 2, shared_xaxes = True,
                        subplot_titles = ["<sup>H1N1 Vaccine</sup>","<sup>H1N1 Vaccine</sup>",
                                        "<sup>Seasonal Vaccine</sup>", "<sup>Seasonal Vaccine</sup>"])

    h1n1 = generate_plot_data(feature, "h1n1_vaccine")
    seasonal = generate_plot_data(feature, "seasonal_vaccine")

    fig1 = px.bar(h1n1, x = feature, y = "count", text_auto = ",",
                color = h1n1["h1n1_vaccine"].astype(str), 
                color_discrete_map = {"0": "#EF553B", "1": "#636EFA"})
    
    fig2 = px.bar(h1n1, x = feature, y = "percent", text_auto = ".1f",
                color = h1n1["h1n1_vaccine"].astype(str),
                color_discrete_map = {"0": "#EF553B", "1": "#636EFA"}).update_traces(showlegend = False)
    
    fig3 = px.bar(seasonal, x = feature, y = "count", text_auto = ",",
                color = seasonal["seasonal_vaccine"].astype(str), 
                color_discrete_map = {"0": "#EF553B", "1": "#636EFA"}).update_traces(showlegend = False)
    
    fig4 = px.bar(seasonal, x = feature, y = "percent", text_auto = ".1f",
                color = seasonal["seasonal_vaccine"].astype(str), 
                color_discrete_map = {"0": "#EF553B", "1": "#636EFA"}).update_traces(showlegend = False)

    for trace in fig1.data:
        fig.add_trace(trace, 1, 1)
    for trace in fig2.data:
        fig.add_trace(trace, 1, 2)
    for trace in fig3.data:
        fig.add_trace(trace, 2, 1)
    for trace in fig4.data:
        fig.add_trace(trace, 2, 2)
    
    fig.update_layout(barmode = "stack", title = {"text": feature.title() + "<br><sup>Histogram & 100% Stacked Bar Chart</sup>", "x": 0.5},
                    legend_title_text = "Vaccinated")
    fig.for_each_trace(lambda t: t.update(name = update_legend[t.name],
                                    legendgroup = update_legend[t.name],
                                    hovertemplate = t.hovertemplate.replace(t.name, update_legend[t.name])))
    if data_dictionary[feature]["mappings"] != {None}:
        update_dict = dict(tickvals = list(data_dictionary[feature]["mappings"].keys()), ticktext = list(data_dictionary[feature]["mappings"].values()))
        fig.update_layout(xaxis3 = update_dict, xaxis4 = update_dict)
    data_distributions.update({feature: fig})

h1n1_concern
h1n1_knowledge
behavioral_antiviral_meds


behavioral_avoidance
behavioral_face_mask
behavioral_wash_hands
behavioral_large_gatherings
behavioral_outside_home
behavioral_touch_face
doctor_recc_h1n1
doctor_recc_seasonal
chronic_med_condition
child_under_6_months
health_worker
health_insurance
opinion_h1n1_vacc_effective
opinion_h1n1_risk
opinion_h1n1_sick_from_vacc
opinion_seas_vacc_effective
opinion_seas_risk
opinion_seas_sick_from_vacc
age_group
education
race
sex
income_poverty
marital_status
rent_or_own
employment_status
hhs_geo_region
census_msa
household_adults
household_children
employment_industry
employment_occupation


In [16]:
# Plotly
# Plot distributions
for tab, var in zip(tabs, features):
    with tab:
        st.write(f"### {var}")
        st.plotly_chart(data_distributions[var])

In [17]:
# LOADED ABOVE
models = {"h1n1_vaccine": {"model": {}, "X_train": {}, "y_train": {}, "X_test": {}, "y_test": {}, "predict": {}, "predict_proba": {}, "model_explainer": {}, "model_performance": {}}, 
          "seasonal_vaccine": {"model": {}, "X_train": {}, "y_train": {}, "X_test": {}, "y_test": {}, "predict": {}, "predict_proba": {}, "model_explainer": {}, "model_performance": {}}}
for response in responses:
    print(response)
    X_train, X_test, y_train, y_test = train_test_split(df.drop(["h1n1_vaccine", "seasonal_vaccine"], axis = 1).select_dtypes("number"), 
                                                        df[response], test_size = 0.20, random_state = 42)
    # https://stackoverflow.com/questions/18689823/pandas-dataframe-replace-nan-values-with-average-of-columns
    X_train.fillna(X_train.median(), inplace = True)
    X_test.fillna(X_test.median(), inplace = True)
    
    mlp = MLPClassifier(early_stopping = True, random_state = 42)
    mlp.fit(X_train, y_train)

    models[response]["model"] = mlp

    models[response]["X_train"] = X_train
    models[response]["y_train"] = y_train
    models[response]["X_test"] = X_test
    models[response]["y_test"] = y_test

    models[response]["predict"] = mlp.predict(X_test)
    models[response]["predict_proba"] = mlp.predict_proba(X_test)
    models[response]["model_explainer"] = dx.Explainer(mlp, X_train, y_train) # Can't save to pickle
    models[response]["model_performance"] = dx.Explainer(mlp, X_train, y_train).model_performance(model_type = "classification")

h1n1_vaccine


Preparation of a new explainer is initiated

  -> data              : 21365 rows 23 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 21365 values
  -> model_class       : sklearn.neural_network._multilayer_perceptron.MLPClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x12d3db420> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.00188, mean = 0.201, max = 0.956
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         :  'residual_function' returns an Error when executed:
'float' object has no attribute 'astype'
  -> model_info        : package sklearn

A new explainer has been created!
Preparation of a new explainer is initiated

  -

In [18]:
plots = {"h1n1_vaccine": {}, "seasonal_vaccine": {}}

In [19]:
for model in models.keys():
    print(f"Target: {model}\n{models[model]['model_performance'].result}")
    plots[model].update({"roc": models[model]["model_performance"].plot(geom = "roc", show = False)})

Target: h1n1_vaccine
                 recall  precision        f1  accuracy       auc
MLPClassifier  0.423415   0.692585  0.525539  0.837398  0.833882


Target: seasonal_vaccine
                 recall  precision        f1  accuracy       auc
MLPClassifier  0.717348     0.7762  0.745614  0.771261  0.844193


In [20]:
protected_vars = ["age_group", "education", "race", "sex", "chronic_med_condition", "income_poverty"]
opinion_vars = ["opinion_h1n1_vacc_effective", "opinion_h1n1_risk", "opinion_h1n1_sick_from_vacc", "opinion_seas_vacc_effective", "opinion_seas_risk", "opinion_seas_sick_from_vacc"]
behaviour_vars = ["behavioral_antiviral_meds", "behavioral_avoidance", "behavioral_face_mask", "behavioral_wash_hands", "behavioral_large_gatherings", "behavioral_outside_home", "behavioral_touch_face"]
health_vars = ["h1n1_concern", "h1n1_knowledge", "doctor_recc_h1n1", "doctor_recc_seasonal", "health_worker", "health_insurance"]
other_vars = ["child_under_6_months", "marital_status", "rent_or_own", "employment_status", "hhs_geo_region", "census_msa", "household_adults", "household_children", "employment_industry", "employment_industry"]

names = ["Protected Characteristics", "Opinion Characteristics", "Behaviour Characteristics", "Health Characteristics", "Other Characteristics"]

In [21]:
# LOADED ABOVE
# Feature importance
for model in models.keys():
    print(model)
    vi = models[model]["model_explainer"].model_parts()
    # vi.result.sort_values(by = "dropout_loss", ascending = False)
    plots[model].update({"vi": vi.plot(title = f"Variable importance: {model}", show = False)})

    for name, vars in zip(names, [protected_vars, opinion_vars, behaviour_vars, health_vars, other_vars]):
        print(name)
        print(vars)
        vars = [x for x in vars if x in X_train.columns]
        print(f"Used in modelling: {vars}")
        other_vars = [x for x in X_train.columns if x not in vars]
        vi_grouped = models[model]["model_explainer"].model_parts(variable_groups = {name: vars, "Remaining Characteristics": other_vars})
        # vi_grouped.result.sort_values(by = "dropout_loss", ascending = False)
        plots[model].update({f"vi_{name}": vi_grouped.plot(title = f"Grouped variable importance: {model}", show = False)})

h1n1_vaccine


Protected Characteristics
['age_group', 'education', 'race', 'sex', 'chronic_med_condition', 'income_poverty']
Used in modelling: ['chronic_med_condition']
Opinion Characteristics
['opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc']
Used in modelling: ['opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc']
Behaviour Characteristics
['behavioral_antiviral_meds', 'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands', 'behavioral_large_gatherings', 'behavioral_outside_home', 'behavioral_touch_face']
Used in modelling: ['behavioral_antiviral_meds', 'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands', 'behavioral_large_gatherings', 'behavioral_outside_home', 'behavioral_touch_face']
Health Characteristics
['h1n1_concern', 'h1n1_know

In [22]:
# LOADED ABOVE
for model in models.keys():
    print(model)
    pdp_num = models[model]["model_explainer"].model_profile(type = "partial", label = "pdp")
    ale_num = models[model]["model_explainer"].model_profile(type = "accumulated", label = "ale")
    plots[model].update({"pdp_ale_num": pdp_num.plot(ale_num, show = False)})

    pdp_cat = models[model]["model_explainer"].model_profile(type = "partial", variable_type = "categorical", variables = ["opinion_seas_risk", "doctor_recc_seasonal"], label = "pdp")
    ale_cat = models[model]["model_explainer"].model_profile(type = "partial", variable_type = "categorical", variables = ["opinion_seas_risk", "doctor_recc_seasonal"], label = "ale")
    plots[model].update({"pdp_ale_cat": pdp_cat.plot(ale_cat, show = False)})

h1n1_vaccine


Calculating ceteris paribus:   0%|          | 0/23 [00:00<?, ?it/s]

Calculating ceteris paribus: 100%|██████████| 2/2 [00:00<00:00,  7.56it/s]


seasonal_vaccine


Calculating ceteris paribus: 100%|██████████| 2/2 [00:00<00:00,  7.85it/s]


In [23]:
# LOADED ABOVE
for model in models.keys():
    print(model)
    print("Most confident 1")
    ptm = time()
    most_confident_1 = np.array(models[model]["X_train"].iloc[np.argmax(models[model]["predict_proba"][:,1])]).T
    bd_1 = models[model]["model_explainer"].predict_parts(most_confident_1, type = "break_down", label = f"{model}: Most Confident Vaccinated Breakdown")
    print(f"1: {time() - ptm:.2f} seconds"); ptm = time()
    bd_interactions_1 = models[model]["model_explainer"].predict_parts(most_confident_1, type = "break_down_interactions", label = f"{model}: Most Confident Vaccinated Interactions Breakdown")
    print(f"2: {time() - ptm:.2f} seconds"); ptm = time()
    sh_1 = models[model]["model_explainer"].predict_parts(most_confident_1, type = "shap", B = 10, label = f"{model}: Most Confident Vaccinated Shap")
    print(f"3: {time() - ptm:.2f} seconds"); ptm = time()

    print("Most confident 0")
    most_confident_0 = np.array(models[model]["X_train"].iloc[np.argmin(models[model]["predict_proba"][:,1])]).T
    bd_0 = models[model]["model_explainer"].predict_parts(most_confident_0, type = "break_down", label = f"{model}: Most Confident Not Vaccinated Breakdown")
    print(f"1: {time() - ptm:.2f} seconds"); ptm = time()
    bd_interactions_0 = models[model]["model_explainer"].predict_parts(most_confident_0, type = "break_down_interactions", label = f"{model}: Most Confident Not Vaccinated Interactions Breakdown")
    print(f"2: {time() - ptm:.2f} seconds"); ptm = time()
    sh_0 = models[model]["model_explainer"].predict_parts(most_confident_0, type = "shap", B = 10, label = f"{model}: Most Confident Not Vaccinated Shap")
    print(f"3: {time() - ptm:.2f} seconds"); ptm = time()

    print("Most indecisive")
    differences = np.abs(models[model]["predict_proba"][:,1] - 0.5)
    most_indecisive = np.array(models[model]["X_train"].iloc[np.argmin(differences),:]).T
    bd_05 = models[model]["model_explainer"].predict_parts(most_indecisive, type = "break_down", label = f"{model}: Most Indecisive Breakdown")
    print(f"1: {time() - ptm:.2f} seconds"); ptm = time()
    bd_interactions_05 = models[model]["model_explainer"].predict_parts(most_indecisive, type = "break_down_interactions", label = f"{model}: Most Indecisive Interactions Breakdown")
    print(f"2: {time() - ptm:.2f} seconds"); ptm = time()
    sh_05 = models[model]["model_explainer"].predict_parts(most_indecisive, type = "shap", B = 10, label = f"{model}: Most Indecisive Shap")
    print(f"3: {time() - ptm:.2f} seconds"); ptm = time()

    # print("CP Values")
    # cp_1 = models[model]["model_explainer"].predict_profile(most_confident_1, label = f"{model}: Most Confident Vaccincated")
    # print(f"1: {time() - ptm:.2f} seconds"); ptm = time()
    # cp_0 = models[model]["model_explainer"].predict_profile(most_confident_0, label = f"{model}: Most Confident Not Vaccinated")
    # print(f"2: {time() - ptm:.2f} seconds"); ptm = time()

    plots[model].update({"bd_1": bd_1.plot(bd_interactions_1, show = False)}); plots[model].update({"sh_1": sh_1.plot(show = False)})
    plots[model].update({"bd_0": bd_0.plot(bd_interactions_0, show = False)}); plots[model].update({"sh_0": sh_0.plot(show = False)})
    plots[model].update({"bd_05": bd_05.plot(bd_interactions_05, show = False)}); plots[model].update({"sh_05": sh_05.plot(show = False)})

    # cp_1.plot(cp_0) # taking way too long (why?)

h1n1_vaccine
Most confident 1


1: 4.00 seconds
2: 21.30 seconds
3: 13.00 seconds
Most confident 0
1: 2.58 seconds
2: 15.21 seconds
3: 10.94 seconds
Most indecisive
1: 2.33 seconds
2: 15.08 seconds
3: 10.41 seconds
seasonal_vaccine
Most confident 1
1: 3.86 seconds
2: 14.69 seconds
3: 10.74 seconds
Most confident 0
1: 2.09 seconds
2: 20.48 seconds
3: 10.44 seconds
Most indecisive
1: 1.84 seconds
2: 15.46 seconds
3: 18.56 seconds


In [24]:
st.write("### Modelling")
st.markdown('''
            The model used for classifying between "Vaccinated" and "Not vaccinated" is a multi-layer Perceptron classifier which optinises the log-loss function using LBFGS or stochastic gradient descent.
            From [scikit-learn: 1.17.1. Multi-layer Perceptron](https://scikit-learn.org/stable/modules/neural_networks_supervised.html):

            \"
            Multi-layer Perceptron (MLP) is a supervised learning algorithm that learns a function $f: \mathrm R^m \\to \mathrm R^o$ by training on a dataset, 
            where $m$ is the number of dimensions for input and $o$ is the number of dimensions for output. Given a set of features $X = x_1, x_2, \ldots, x_m$ and a target $y$, 
            it can learn a non-linear function approximator for either classification or regression. It is different from logistic regression, 
            in that between the input and the output layer, there can be one or more non-linear layers, called hidden layers. 

            The leftmost layer, known as the input layer, consists of a set of neurons $\{x_i | x_1, x_2, \ldots, x_m\}$
            representing the input features. Each neuron in the hidden layer transforms the values from the previous layer with a weighted linear summation
            $w_1 x_1 + w_2 x_2 + \ldots + w_m x_m $, 
            followed by a non-linear activation function $g(\cdot): \mathrm R \\to \mathrm R$ - like the hyperbolic tan function. 
            The output layer receives the values from the last hidden layer and transforms them into output values.
            \"
         ''')


tab36, tab37 = st.tabs(["H1N1 Vaccine", "Seasonal Vaccine"])
tab38, tab39, tab40, tab41, tab42, tab43 = st.tabs(["Model Performance", "Model Explanations", "Model Fairnesss"] * 2)

# tabs_iter = [(tab36, tab38), (tab36, tab39), (tab36, tab40), 
#              (tab37, tab41), (tab37, tab42), (tab37, tab43)]

# # Model Performance
# for model_tab, sub_tab in zip(tabs, features):
#     with model_tab:
#         with tab:

In [25]:
model = "h1n1_vaccine"

with tab36:
    with tab38:
        fig, ax = plt.subplots()
        ax.hist(models[model]["predict_proba"][:,1], color = "skyblue", edgecolor = "black")
        ax.set_title("MLP Classifier: Distribution of Predicted Values")
        st.pyplot(fig)

        # MLP
        st.write("MLP:\n\nClassification Report:")
        st.dataframe(pd.DataFrame(classification_report(models[model]["y_test"], models[model]["predict"], output_dict = True)).T)

        confusion_matrix_explained = st.expander("Confusion Matrix Explained")
        with confusion_matrix_explained:
            st.markdown('''
                        In the field of machine learning and specifically the problem of statistical classification, a confusion matrix, also known as error matrix, is a specific table layout that allows visualization of the performance of an algorithm, typically a supervised learning one; in unsupervised learning it is usually called a matching matrix.
                        Each row of the matrix represents the instances in an actual class while each column represents the instances in a predicted class, or vice versa – both variants are found in the literature. The name stems from the fact that it makes it easy to see whether the system is confusing two classes (i.e. commonly mislabeling one as another).
                        It is a special kind of contingency table, with two dimensions ("actual" and "predicted"), and identical sets of "classes" in both dimensions (each combination of dimension and class is a variable in the contingency table).
                        
                        [source](https://en.wikipedia.org/wiki/Confusion_matrix)
                        ''')

        fig, ax = plt.subplots()
        ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(models[model]["y_test"], models[model]["predict"])).plot(ax = ax)
        ax.set_title("Confusion Matrix")
        st.pyplot(fig)

        roc_curve_explained = st.expander("ROC Curve Explained")
        with roc_curve_explained:
            st.markdown('''
                        A receiver operating characteristic curve, or ROC curve, is a graphical plot that illustrates the performance of a binary classifier model (can be used for multi class classification as well) at varying threshold values.
                        The ROC curve is the plot of the true positive rate (TPR) against the false positive rate (FPR) at each threshold setting.
                        The ROC can also be thought of as a plot of the statistical power as a function of the Type I Error of the decision rule (when the performance is calculated from just a sample of the population, it can be thought of as estimators of these quantities). The ROC curve is thus the sensitivity or recall as a function of false positive rate.
                        Given the probability distributions for both true positive and false positive are known, the ROC curve is obtained as the cumulative distribution function (CDF, area under the probability distribution from 
                        $- \infty$ to the discrimination threshold) of the detection probability in the y-axis versus the CDF of the false positive probability on the x-axis.
                        ROC analysis provides tools to select possibly optimal models and to discard suboptimal ones independently from (and prior to specifying) the cost context or the class distribution. ROC analysis is related in a direct and natural way to the cost/benefit analysis of diagnostic decision making.
                        
                        [source](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)
                        ''')
            
        fig, ax = plt.subplots()
        fpr, tpr, thresholds = roc_curve(models[model]["y_test"], models[model]["predict_proba"][:,1]) 
        roc_auc = auc(fpr, tpr)
        # Plot the ROC curve 
        ax.plot(fpr, tpr, label = f"ROC curve (area = {roc_auc:0.2f})")
        ax.plot([0, 1], [0, 1], 'k--', label='No Skill')
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title('ROC Curve')
        ax.legend()
        st.pyplot(fig)

    # Model Explanations
    with tab39:
        st.write("#### Variable Importance")
        
        feature_importance_explained = st.expander("Feature Importance Explained")
        with feature_importance_explained:
            st.markdown('''
                        Feature importance is a step in building a machine learning model that involves calculating the score for all input features in a model to establish the importance of each feature in the decision-making process. 
                        The higher the score for a feature, the larger effect it has on the model to predict a certain variable. 
                        
                        [source](https://builtin.com/data-science/feature-importance#:~:text=Feature%20importance%20is%20a%20step,to%20predict%20a%20certain%20variable.)
                        ''')

        st.plotly_chart(plots[model]["vi"])
        st.markdown('''
                    Here, variable importance is computed by the drop-out loss. 

                    $$\\text{Dropout Loss} = \\text{Performance with all variables} − \\text{Performance with variable dropped}$$

                    A larger dropout loss indicates that the variable is more important because its removal significantly impacts the model's performance.

                    As we can see from the variable importance plot, our top 5 most 'influential' features in the model are: `GenHealth`, `BMI`, `Age`, `HighBP`, `HighChol`.

                    The least 'influential' features in the model are: `PhysActivity`, `Veggies`, `AnyHealthcare`, `Education`, `NoDocbcCost`.
                    ''')

        st.plotly_chart(plots[model]["vi_Protected Characteristics"])
        st.plotly_chart(plots[model]["vi_Opinion Characteristics"])
        st.plotly_chart(plots[model]["vi_Behaviour Characteristics"])
        st.plotly_chart(plots[model]["vi_Health Characteristics"])
        st.plotly_chart(plots[model]["vi_Other Characteristics"])
        st.markdown('''
                    Here, we also want to assess the variable importance of features when they are grouped. Most interestingly, we can look to see how protected characteristics (i.e. `Sex`, `Age`, `Education`, `Income`) impact a model's performance. This is to explore if they can be dropped fully or not with any significant changes to the mode's predictive ability.
                    
                    As we can see from the plot above, protected characteristics have limited influence over the model's performance when permuted from the feature list (compared to 'unprotected' charaxcteristics). Therefore, it can be said that dropping these features will be of limited consequence for the accuracy of the model, whilst improving other statistics (such as when we perform fairness checks).
                    ''')

        st.write("#### Partial Dependence & Accumulated Local Dependence")
        
        partial_dependence_explained = st.expander("Partial Dependence & Accumulated Local Dependence Explained")
        with partial_dependence_explained:
            st.markdown('''
                        The general idea underlying the construction of Partial-dependence (PD) profiles is to show how does the expected value of model prediction behave as a function of a selected explanatory variable. For a single model, one can construct an overall PD profile by using all observations from a dataset, or several profiles for sub-groups of the observations. Comparison of sub-group-specific profiles may provide important insight into, for instance, the stability of the model’s predictions.
                        PD profiles are easy to explain and interpret, especially given their estimation as the mean of ceteris-paribus (CP) profiles. However, the profiles may be misleading if, for instance, explanatory variables are correlated (in ceteris-paribus profiles).
                        Accumulated-local profiles address this issue, and provides a more refined understanding compared to traditional partial dependence plots by focusing on local changes rather than averaging over the entire dataset.

                        [source 1](https://ema.drwhy.ai/partialDependenceProfiles.html)

                        [source 2](https://ema.drwhy.ai/accumulatedLocalProfiles.html)
                        ''')

        st.plotly_chart(plots[model]["pdp_ale_num"])

        st.markdown('''
                    Explain
                    ''')

        st.plotly_chart(plots[model]["pdp_ale_cat"])

        st.markdown('''
                    Explain
                    ''')

        st.write("#### Prediction Breakdown & Shapley Values")

        shapley_values_explained = st.expander("Shapley Values Explained")
        with shapley_values_explained:
            st.markdown('''
                        In explainable machine learning the Shapley value is used to measure the contributions of input features to the output of a machine learning model at the instance level. Given a specific data point, the goal is to decompose the model prediction and assign Shapley values to individual features of the instance.

                        [source](https://www.ijcai.org/proceedings/2022/0778.pdf)
                        ''')

        st.write("#### Most confident model prediction")
        st.write("#### Positive Class (i.e. Vaccinated = 1)")

        st.plotly_chart(plots[model]["bd_1"])
        st.markdown('''
                    Explain
                    ''')

        st.plotly_chart(plots[model]["sh_1"])
        st.markdown('''
                    Explain
                    ''')

        st.write("#### Negative Class (i.e. Not vaccinated = 0)")
        
        st.plotly_chart(plots[model]["bd_0"])
        st.markdown('''
                    Explain
                    ''')

        st.plotly_chart(plots[model]["sh_0"])
        st.markdown('''
                    Explain
                    ''')
        
        st.write("#### Most indecesive model prediction")
        
        st.plotly_chart(plots[model]["bd_05"])
        st.markdown('''
                    Explain
                    ''')

        st.plotly_chart(plots[model]["sh_05"])
        st.markdown('''
                    Explain
                    ''')

        st.write("#### Ceteris Paribus Profiles for Most Confident Predictions")

        ceteris_paribus_explained = st.expander("Ceteris Paribus Profiles Explained")
        with ceteris_paribus_explained:
            st.markdown('''
                        Ceteris-paribus (CP) profiles show how a model’s prediction would change if the value of a single exploratory variable changed. In essence, a CP profile shows the dependence of the conditional expectation of the dependent variable (response) on the values of the particular explanatory variable.

                        [source](https://ema.drwhy.ai/ceterisParibus.html)
                        ''')

        # st.plotly_chart(cp_1.plot(cp_0, show = False))
        st.markdown('''
                    Explain
                    ''')

    # Model Fairness
    with tab40:

        df_preds = pd.concat([
            models[model]["X_test"].reset_index(drop = True),
            models[model]["y_test"].reset_index(drop = True),
            pd.DataFrame(models[model]["predict"], columns = ["predict"]).reset_index(drop = True)],
            axis = 1)
        # fairness_var = st.radio("Select protected characteristic", ["Age", "Sex", "Education", "Income"])
        fairness_sex_0 = ff.group_fairness(df_preds, "chronic_med_condition", 0, "predict", 1)
        fairness_sex_1 = ff.group_fairness(df_preds, "chronic_med_condition", 1, "predict", 1)

        fairness_risk_sex_0 = ff.conditional_statistical_parity(df_preds, "chronic_med_condition", 0, "predict", 1, "opinion_h1n1_risk", 1)
        fairness_risk_sex_1 = ff.conditional_statistical_parity(df_preds, "chronic_med_condition", 1, "predict", 1, "opinion_h1n1_risk", 1)

        parity_sex_0 = ff.predictive_parity(df_preds, "chronic_med_condition", 0, "predict", model)
        parity_sex_1 = ff.predictive_parity(df_preds, "chronic_med_condition", 1, "predict", model)

        er_sex_0 = ff.fp_error_rate_balance(df_preds, "chronic_med_condition", 0, "predict", model)
        er_sex_1 = ff.fp_error_rate_balance(df_preds, "chronic_med_condition", 1, "predict", model)

        st.write("### Fairness Metrics")

        st.write("#### Group Fairness")
        st.write("Members of each group need to have the same probability of being assigned to the positively predicted class.")
        st.write(f"Chronic Medical Condition class 0 (no):\t{fairness_sex_0}")
        st.write(f"Chronic Medical Condition class 1 (yes):\t{fairness_sex_1}")
        st.write(f"Difference: {np.abs(fairness_sex_0 - fairness_sex_1):.3f}")

        st.write("#### Conditional Statistical Parity")
        st.write("Members of each group need to have the same probability of being assigned to the positive class under the same set of conditions.")
        st.write(f"Opinion H1N1 Risk +Chronic Medical Condition class 0 (no):\t{fairness_risk_sex_0}")
        st.write(f"Opinion H1N1 Risk + Chronic Medical Condition class 1 (yes):\t{fairness_risk_sex_1}")
        st.write(f"Difference: {np.abs(fairness_risk_sex_0 - fairness_risk_sex_1):.3f}")

        st.write(f"#### Predictive Parity")
        st.write("Members of each group have the same Positive Predictive Value (PPV) — the probability of a subject with Positive Predicted Value to truly belong to the positive class.")
        st.write(f"Chronic Medical Condition class 0 (no):\t{parity_sex_0:.3f}")
        st.write(f"Chronic Medical Condition class 1 (yes):\t{parity_sex_1:.3f}")
        st.write(f"Difference: {np.abs(parity_sex_0 - parity_sex_1):.3f}")

        st.write(f"#### False Positive Error Rate Balance")
        st.write("Members of each group have the same False Positive Rate (FPR) — the probability of a subject in the negative class to have a positive predicted value.")
        st.write(f"Chronic Medical Condition class 0 (no):\t{er_sex_0:.3f}")
        st.write(f"Chronic Medical Condition class 1 (yes):\t{er_sex_1:.3f}")
        st.write(f"Difference: {np.abs(er_sex_0 - er_sex_1):.3f}")

In [26]:
model = "seasonal_vaccine"

with tab37:
    with tab41:
        fig, ax = plt.subplots()
        ax.hist(models[model]["predict_proba"][:,1], color = "skyblue", edgecolor = "black")
        ax.set_title("MLP Classifier: Distribution of Predicted Values")
        st.pyplot(fig)

        # MLP
        st.write("MLP:\n\nClassification Report:")
        st.dataframe(pd.DataFrame(classification_report(models[model]["y_test"], models[model]["predict"], output_dict = True)).T)

        confusion_matrix_explained = st.expander("Confusion Matrix Explained")
        with confusion_matrix_explained:
            st.markdown('''
                        In the field of machine learning and specifically the problem of statistical classification, a confusion matrix, also known as error matrix, is a specific table layout that allows visualization of the performance of an algorithm, typically a supervised learning one; in unsupervised learning it is usually called a matching matrix.
                        Each row of the matrix represents the instances in an actual class while each column represents the instances in a predicted class, or vice versa – both variants are found in the literature. The name stems from the fact that it makes it easy to see whether the system is confusing two classes (i.e. commonly mislabeling one as another).
                        It is a special kind of contingency table, with two dimensions ("actual" and "predicted"), and identical sets of "classes" in both dimensions (each combination of dimension and class is a variable in the contingency table).
                        
                        [source](https://en.wikipedia.org/wiki/Confusion_matrix)
                        ''')

        fig, ax = plt.subplots()
        ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(models[model]["y_test"], models[model]["predict"])).plot(ax = ax)
        ax.set_title("Confusion Matrix")
        st.pyplot(fig)

        roc_curve_explained = st.expander("ROC Curve Explained")
        with roc_curve_explained:
            st.markdown('''
                        A receiver operating characteristic curve, or ROC curve, is a graphical plot that illustrates the performance of a binary classifier model (can be used for multi class classification as well) at varying threshold values.
                        The ROC curve is the plot of the true positive rate (TPR) against the false positive rate (FPR) at each threshold setting.
                        The ROC can also be thought of as a plot of the statistical power as a function of the Type I Error of the decision rule (when the performance is calculated from just a sample of the population, it can be thought of as estimators of these quantities). The ROC curve is thus the sensitivity or recall as a function of false positive rate.
                        Given the probability distributions for both true positive and false positive are known, the ROC curve is obtained as the cumulative distribution function (CDF, area under the probability distribution from 
                        $- \infty$ to the discrimination threshold) of the detection probability in the y-axis versus the CDF of the false positive probability on the x-axis.
                        ROC analysis provides tools to select possibly optimal models and to discard suboptimal ones independently from (and prior to specifying) the cost context or the class distribution. ROC analysis is related in a direct and natural way to the cost/benefit analysis of diagnostic decision making.
                        
                        [source](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)
                        ''')
            
        fig, ax = plt.subplots()
        fpr, tpr, thresholds = roc_curve(models[model]["y_test"], models[model]["predict_proba"][:,1]) 
        roc_auc = auc(fpr, tpr)
        # Plot the ROC curve 
        ax.plot(fpr, tpr, label = f"ROC curve (area = {roc_auc:0.2f})")
        ax.plot([0, 1], [0, 1], 'k--', label='No Skill')
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title('ROC Curve')
        ax.legend()
        st.pyplot(fig)

    # Model Explanations
    with tab42:
        st.write("#### Variable Importance")
        
        feature_importance_explained = st.expander("Feature Importance Explained")
        with feature_importance_explained:
            st.markdown('''
                        Feature importance is a step in building a machine learning model that involves calculating the score for all input features in a model to establish the importance of each feature in the decision-making process. 
                        The higher the score for a feature, the larger effect it has on the model to predict a certain variable. 
                        
                        [source](https://builtin.com/data-science/feature-importance#:~:text=Feature%20importance%20is%20a%20step,to%20predict%20a%20certain%20variable.)
                        ''')

        st.plotly_chart(plots[model]["vi"])
        st.markdown('''
                    Here, variable importance is computed by the drop-out loss. 

                    $$\\text{Dropout Loss} = \\text{Performance with all variables} − \\text{Performance with variable dropped}$$

                    A larger dropout loss indicates that the variable is more important because its removal significantly impacts the model's performance.

                    As we can see from the variable importance plot, our top 5 most 'influential' features in the model are: `GenHealth`, `BMI`, `Age`, `HighBP`, `HighChol`.

                    The least 'influential' features in the model are: `PhysActivity`, `Veggies`, `AnyHealthcare`, `Education`, `NoDocbcCost`.
                    ''')
        
        st.plotly_chart(plots[model]["vi_Protected Characteristics"])
        st.plotly_chart(plots[model]["vi_Opinion Characteristics"])
        st.plotly_chart(plots[model]["vi_Behaviour Characteristics"])
        st.plotly_chart(plots[model]["vi_Health Characteristics"])
        st.plotly_chart(plots[model]["vi_Other Characteristics"])
        st.markdown('''
                    Here, we also want to assess the variable importance of features when they are grouped. Most interestingly, we can look to see how protected characteristics (i.e. `Sex`, `Age`, `Education`, `Income`) impact a model's performance. This is to explore if they can be dropped fully or not with any significant changes to the mode's predictive ability.
                    
                    As we can see from the plot above, protected characteristics have limited influence over the model's performance when permuted from the feature list (compared to 'unprotected' charaxcteristics). Therefore, it can be said that dropping these features will be of limited consequence for the accuracy of the model, whilst improving other statistics (such as when we perform fairness checks).
                    ''')

        st.write("#### Partial Dependence & Accumulated Local Dependence")
        
        partial_dependence_explained = st.expander("Partial Dependence & Accumulated Local Dependence Explained")
        with partial_dependence_explained:
            st.markdown('''
                        The general idea underlying the construction of Partial-dependence (PD) profiles is to show how does the expected value of model prediction behave as a function of a selected explanatory variable. For a single model, one can construct an overall PD profile by using all observations from a dataset, or several profiles for sub-groups of the observations. Comparison of sub-group-specific profiles may provide important insight into, for instance, the stability of the model’s predictions.
                        PD profiles are easy to explain and interpret, especially given their estimation as the mean of ceteris-paribus (CP) profiles. However, the profiles may be misleading if, for instance, explanatory variables are correlated (in ceteris-paribus profiles).
                        Accumulated-local profiles address this issue, and provides a more refined understanding compared to traditional partial dependence plots by focusing on local changes rather than averaging over the entire dataset.

                        [source 1](https://ema.drwhy.ai/partialDependenceProfiles.html)

                        [source 2](https://ema.drwhy.ai/accumulatedLocalProfiles.html)
                        ''')

        st.plotly_chart(plots[model]["pdp_ale_num"])
        st.markdown('''
                    Explain
                    ''')

        st.plotly_chart(plots[model]["pdp_ale_cat"])
        st.markdown('''
                    Explain
                    ''')

        st.write("#### Prediction Breakdown & Shapley Values")

        shapley_values_explained = st.expander("Shapley Values Explained")
        with shapley_values_explained:
            st.markdown('''
                        In explainable machine learning the Shapley value is used to measure the contributions of input features to the output of a machine learning model at the instance level. Given a specific data point, the goal is to decompose the model prediction and assign Shapley values to individual features of the instance.

                        [source](https://www.ijcai.org/proceedings/2022/0778.pdf)
                        ''')

        st.write("#### Most confident model prediction")
        st.write("#### Positive Class (i.e. Vaccinated = 1)")

        st.plotly_chart(plots[model]["bd_1"])
        st.markdown('''
                    Explain
                    ''')

        st.plotly_chart(plots[model]["sh_1"])
        st.markdown('''
                    Explain
                    ''')

        st.write("#### Negative Class (i.e. Not vaccinated = 0)")
        
        st.plotly_chart(plots[model]["bd_0"])
        st.markdown('''
                    Explain
                    ''')

        st.plotly_chart(plots[model]["sh_0"])
        st.markdown('''
                    Explain
                    ''')
        
        st.write("#### Most indecesive model prediction")
        
        st.plotly_chart(plots[model]["bd_05"])
        st.markdown('''
                    Explain
                    ''')

        st.plotly_chart(plots[model]["sh_05"])
        st.markdown('''
                    Explain
                    ''')

        st.write("#### Ceteris Paribus Profiles for Most Confident Predictions")

        ceteris_paribus_explained = st.expander("Ceteris Paribus Profiles Explained")
        with ceteris_paribus_explained:
            st.markdown('''
                        Ceteris-paribus (CP) profiles show how a model’s prediction would change if the value of a single exploratory variable changed. In essence, a CP profile shows the dependence of the conditional expectation of the dependent variable (response) on the values of the particular explanatory variable.

                        [source](https://ema.drwhy.ai/ceterisParibus.html)
                        ''')

        # st.plotly_chart(cp_1.plot(cp_0, show = False))
        st.markdown('''
                    Explain
                    ''')

    # Model Fairness
    with tab43:

        df_preds = pd.concat([
            models[model]["X_test"].reset_index(drop = True),
            models[model]["y_test"].reset_index(drop = True),
            pd.DataFrame(models[model]["predict"], columns = ["predict"]).reset_index(drop = True)],
            axis = 1)
        # fairness_var = st.radio("Select protected characteristic", ["Age", "Sex", "Education", "Income"])
        fairness_sex_0 = ff.group_fairness(df_preds, "chronic_med_condition", 0, "predict", 1)
        fairness_sex_1 = ff.group_fairness(df_preds, "chronic_med_condition", 1, "predict", 1)

        fairness_risk_sex_0 = ff.conditional_statistical_parity(df_preds, "chronic_med_condition", 0, "predict", 1, "opinion_seas_risk", 1)
        fairness_risk_sex_1 = ff.conditional_statistical_parity(df_preds, "chronic_med_condition", 1, "predict", 1, "opinion_seas_risk", 1)

        parity_sex_0 = ff.predictive_parity(df_preds, "chronic_med_condition", 0, "predict", model)
        parity_sex_1 = ff.predictive_parity(df_preds, "chronic_med_condition", 1, "predict", model)

        er_sex_0 = ff.fp_error_rate_balance(df_preds, "chronic_med_condition", 0, "predict", model)
        er_sex_1 = ff.fp_error_rate_balance(df_preds, "chronic_med_condition", 1, "predict", model)

        st.write("### Fairness Metrics")

        st.write("#### Group Fairness")
        st.write("Members of each group need to have the same probability of being assigned to the positively predicted class.")
        st.write(f"Chronic Medical Condition class 0 (no):\t{fairness_sex_0}")
        st.write(f"Chronic Medical Condition class 1 (yes):\t{fairness_sex_1}")
        st.write(f"Difference: {np.abs(fairness_sex_0 - fairness_sex_1):.3f}")

        st.write("#### Conditional Statistical Parity")
        st.write("Members of each group need to have the same probability of being assigned to the positive class under the same set of conditions.")
        st.write(f"Opinion Seasonal Risk +Chronic Medical Condition class 0 (no):\t{fairness_risk_sex_0}")
        st.write(f"Opinion Seasonal Risk + Chronic Medical Condition class 1 (yes):\t{fairness_risk_sex_1}")
        st.write(f"Difference: {np.abs(fairness_risk_sex_0 - fairness_risk_sex_1):.3f}")

        st.write(f"#### Predictive Parity")
        st.write("Members of each group have the same Positive Predictive Value (PPV) — the probability of a subject with Positive Predicted Value to truly belong to the positive class.")
        st.write(f"Chronic Medical Condition class 0 (no):\t{parity_sex_0:.3f}")
        st.write(f"Chronic Medical Condition class 1 (yes):\t{parity_sex_1:.3f}")
        st.write(f"Difference: {np.abs(parity_sex_0 - parity_sex_1):.3f}")

        st.write(f"#### False Positive Error Rate Balance")
        st.write("Members of each group have the same False Positive Rate (FPR) — the probability of a subject in the negative class to have a positive predicted value.")
        st.write(f"Chronic Medical Condition class 0 (no):\t{er_sex_0:.3f}")
        st.write(f"Chronic Medical Condition class 1 (yes):\t{er_sex_1:.3f}")
        st.write(f"Difference: {np.abs(er_sex_0 - er_sex_1):.3f}")

In [28]:
del models["h1n1_vaccine"]["model_explainer"]
del models["seasonal_vaccine"]["model_explainer"]

In [29]:
with open("data_distributions.pkl", "wb") as file:
    # Serialize and save the object to the file
    pickle.dump(data_distributions, file)
with open("models.pkl", "wb") as file:
    # Serialize and save the object to the file
    pickle.dump(models, file)
with open("plots.pkl", "wb") as file:
    # Serialize and save the object to the file
    pickle.dump(plots, file)